In [1]:
import tkinter as tk
import numpy as np
import time
import random
import copy
import math

# Set number of rows and columns
ROWS = 250
COLS = 250
WIDTH = 100
HEIGHT = 100
ANIMATION_DELAY = 50
NEIGHBOUR_SEARCH_DELAY = 0
BORDER_PROB = 0.2
ITERATIONS_CONWAY = 1000


# Create a grid of None to store the references to the tiles
cells = [[None for _ in range(ROWS)] for _ in range(COLS)]
pixels = [[None for _ in range(HEIGHT)] for _ in range(WIDTH)]
clicks = []

class Cell:

    def __init__(self, col, row):
        self.col = col
        self.row = row
        self.rect = None
        self.text = None
        createRectangle(self,"","","","")
        # extras for A_Star:
        self.cost_g = 0
        self.cost = 0
        self.parent = None
        self.dist_to_neighbours = []
    
    def equals(self,cell):
        return (self.col == cell.col and self.row == cell.row)
    
    def changeText(self,txt,color):
        c.itemconfig(self.text,text=txt,fill=color)
        
    def changeRect(self,color):
        c.itemconfig(self.rect, fill=color)
    
    def toString(self):
        return str(self.col) + "|" + str(self.row)
    
    # extras for A_Star:
    def getDist(self,cell):
        return 1
    

class Pixel:

    def __init__(self, col, row):
        self.col = col
        self.row = row
        self.line = c.create_line(col,row,col+1,row,fill="")
    
    def equals(self,cell):
        return (self.col == cell.col and self.row == cell.row)
    
    def changeColor(self,color):
        c.itemconfig(self.line, fill=color)
    
    def toString(self):
        return str(self.col) + "|" + str(self.row)
     
def callback2(event):
    # Calculate column and row number
    col = int(event.x/col_width)
    row = int(event.y/row_height)
    clicked_cell = cells[col][row]
    # If the tile is not filled, create a rectangle
    if not c.itemcget(clicked_cell.rect,"fill") == "black":
        clicked_cell.changeRect("black")
    else:
        startConway(ITERATIONS_CONWAY)
            
def callback3(event):
    global clicks
    global mandelbrot
    # Calculate column and row number
    col = event.x
    row = event.y
    #print(clicked_cell.toString())
    #print()
    if len(clicks) < 1: 
        clicks.append([((mandelbrot.x_max-mandelbrot.x_min)/WIDTH)*col+mandelbrot.x_min, ((mandelbrot.y_min-mandelbrot.y_max)/HEIGHT)*row+mandelbrot.y_max])
    else:
        clicks.append([((mandelbrot.x_max-mandelbrot.x_min)/WIDTH)*col+mandelbrot.x_min, ((mandelbrot.y_min-mandelbrot.y_max)/HEIGHT)*row+mandelbrot.y_max])
        print("Upper Corner: ", str(clicks[0][0]),str(clicks[0][1]))
        print("Lower Corner: ", str(clicks[1][0]),str(clicks[1][1]))
        mandelbrot.changeView(clicks[0][0],clicks[1][0],clicks[1][1], clicks[0][1])
        mandelbrot.draw()
        clicks = []
    
def startConway(iteration):
    
    if iteration == 0:
        return
    
    alive = np.zeros([COLS,ROWS])
    
    t0 = time.time()
    
    for row in range(ROWS):
        for col in range(COLS):
            current_cell = cells[col][row]
            neighbours = getAliveNeighbours(current_cell)
            isAlive = c.itemcget(current_cell.rect,"fill") == "black"
            
            willSurvive = isAlive and (neighbours == 2 or neighbours == 3)
            willResurrect = not isAlive and neighbours == 3
            
            if willSurvive or willResurrect:
                alive[col][row] = 1
            else:
                alive[col][row] = 0
            
    t1 = time.time()
    
    for row in range(ROWS):
        for col in range(COLS):
            current_cell = cells[col][row]
            isAlive = alive[col][row]
            
            if isAlive:
                current_cell.changeRect("black")
            else:
                current_cell.changeRect("white")
                
    t2 = time.time()
    
    root.after(ANIMATION_DELAY,startConway,iteration-1)
       
    total = t1-t0
    
    print("neighbour Loop: ", t1-t0)
    print("draw Loop: ", t2-t1)
    print(len(c.winfo_children()))
    print("----------------------")
    
def getAliveNeighbours(coord):
    count = 0
    
    upper_left = Cell(coord.col - 1,coord.row - 1) 
    neighbours = []
    for row in range(3):
        for col in range(3):
            new_col = (upper_left.col + col) % COLS
            new_row = (upper_left.row + row) % ROWS
            
            isStartingCell = (row == 1 and col == 1)
            isAlive = c.itemcget(cells[new_col][new_row].rect,"fill") == "black"
            
            if not isStartingCell and isAlive:
                count = count + 1
                
    return count

def createRectangle(cell,rect_color,rect_outline,txt,text_color):
        cell.rect = c.create_rectangle(cell.col*col_width, cell.row*row_height, (cell.col+1)*col_width, (cell.row+1)*row_height, fill=rect_color, outline=rect_outline)
        cell.text = c.create_text(((cell.col*col_width + (cell.col+1)*col_width)/2, (cell.row*row_height + (cell.row+1)*row_height)/2), text=txt, fill=text_color)

def initializeCanvas2(hasBorder):
    print("hghfhg")
    row = 20
    for col in range(COLS):
        cells[col][row].changeRect("black")
    print("bbbhb")
    row = 22
    for col in range(COLS):
        cells[col][row].changeRect("black")        


def initializeCanvas(hasBorder):
    for row in range(ROWS):
        for col in range(COLS):
            cells[col][row] = Cell(col,row)
            if hasBorder and (row == 0 or col == 0 or col == COLS-1 or row == ROWS-1):
                cells[col][row].changeRect("black")
                
    for row in range(HEIGHT):
        for col in range(WIDTH):
            pixels[col][row] = Pixel(col,row)
        
class ComplexNumber:
    
    def __init__(self,real,imaginary):
        self.real = real
        self.imaginary = imaginary
    
    def squared(self):
        return ComplexNumber(self.real**2-self.imaginary**2,2*self.real*self.imaginary)
    
    def abs(self):
        return math.sqrt(self.real**2+self.imaginary**2)
    
    def add(self,c):
        return ComplexNumber(self.real+c.real,self.imaginary+c.imaginary)
    
    def toString(self):
        return "("+str(self.real)+", "+str(self.imaginary)+"i)"
        

class Mandelbrot:
    
    def __init__(self, x_min=-2.2, x_max=0.8, y_min = -1, y_max= 1, MAX_ABS = 1000, MAX_ITER = 70):
        self.x_min = x_min 
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.MAX_ABS = MAX_ABS
        self.MAX_ITER = MAX_ITER
        
    def changeView(self, x_min, x_max, y_min, y_max):
        self.x_min = x_min 
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
   
    def draw(self):

        for row in range(HEIGHT):
            for col in range(WIDTH):
                z = ComplexNumber(0,0)
                #com = ComplexNumber(((self.x_max-self.x_min)/COLS)*col+self.x_min,((self.y_min-self.y_max)/ROWS)*row+self.y_max)
                com = ComplexNumber(((self.x_max-self.x_min)/WIDTH)*col+self.x_min,((self.y_min-self.y_max)/HEIGHT)*row+self.y_max)
                #print("Test "+c.toString())
                n = 0

                while n < self.MAX_ITER:
                    #print(str(n),": ",z.toString()," | ",str(z.abs()))
                    if z.abs() > self.MAX_ABS:
                        #cells[col][row].changeText(str(n),"black")
                        color = "#{:02x}".format(abs(int(self.linReg(255,-255,7*self.MAX_ITER,n)))) + "{:02x}".format(int(self.quadReg(255,self.MAX_ITER,n))) + "{:02x}".format(int(abs(self.linReg(255,-255,self.MAX_ITER,n))*self.linReg(0,1,self.MAX_ITER,n)))
                        #cells[col][row].changeRect(color)
                        pixels[col][row].changeColor(color)
                        break
                    z = z.squared().add(com)
                    n = n + 1

                #print("---------------------------")
                if n == self.MAX_ITER:
                    pixels[col][row].changeColor("black")
                    #cells[col][row].changeRect("black")
                    #cells[col][row].changeText(c.toString(),"white")
        
    def linReg(self,start,end,t_max,t):
        c = start
        m = (end-start)/t_max
        return m*t+c

    def quadReg(self,y_mid,t_max,t):
        c = 0
        a = (-4*y_mid)/(t_max*t_max)
        b = -a*t_max
        return a*(t**2)+b*t+c

                    

    
                
root = tk.Tk()

c = tk.Canvas(root, width=WIDTH, height=HEIGHT, borderwidth=5, background='white')
c.pack()
c.bind("<Button-1>", callback3)
c.update()

col_width = c.winfo_width()/COLS
row_height = c.winfo_height()/ROWS
initializeCanvas(False)
#initializeCanvas2(False)
#mandelbrot = Mandelbrot(0.26,0.38,0.32,0.44)
mandelbrot = Mandelbrot()

mandelbrot.draw()
print(col_width)
print(row_height)

root.mainloop()

0.456
0.456
Upper Corner:  -0.43000000000000016 -0.6200000000000001
Lower Corner:  0.19999999999999973 -0.9199999999999999
Upper Corner:  -0.20320000000000019 -0.845
Lower Corner:  -0.014200000000000212 -0.9019999999999999
Upper Corner:  -0.1351600000000002 -0.88433
Lower Corner:  -0.06901000000000021 -0.8991499999999999
Upper Corner:  -0.1053925000000002 -0.893963
Lower Corner:  -0.08687050000000021 -0.8963341999999999
Upper Corner:  -0.1024289800000002 -0.894294968
Lower Corner:  -0.0939088600000002 -0.8951011759999999
Upper Corner:  -0.0975725116000002 -0.89489156192
Lower Corner:  -0.0959536888000002 -0.8950286172799999
Upper Corner:  -0.0963907709560002 -0.8949546073856
Lower Corner:  -0.0961479475360002 -0.8949888712256
Upper Corner:  -0.0963300651010002 -0.8949659144527999
Lower Corner:  -0.0962110816252002 -0.8949734524976
Upper Corner:  -0.0963038887363242 -0.894968402007584
Lower Corner:  -0.0962182206337482 -0.894970286518784
